In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from data_preprocessing import IsriDataset
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score, silhouette_score

#GA_SOLUTIONS_PATH = "IsriDataset.pkl"
GA_SOLUTIONS_PATH = "IsriDataDict.pkl"

isri_dataset = pickle.load(open(GA_SOLUTIONS_PATH, 'rb'))

df = pd.DataFrame()
for i in range(len(isri_dataset.data['Jobdata'])):
    df_temp = pd.DataFrame.from_dict(isri_dataset.data['Jobdata'][i], orient='index')
    df = pd.concat([df, df_temp])

df[['time1','time2','time3','time4','time5','time6','time7','time8','time9','time10','time11','time12']] = pd.DataFrame(df.times.tolist(), index= df.index)
#print(df)
# 'time11',

df_cluster = df[['time1','time2','time3','time4','time5','time6','time7','time8','time9','time10','time12']] #due_date
#df_cluster.to_excel('output.xlsx', index=False)
#pd.plotting.scatter_matrix(df_cluster, alpha=0.2, figsize=(15,15))
#plt.show()




AttributeError: 'dict' object has no attribute 'data'

In [ ]:
kmeans = KMeans(n_clusters=8).fit(df_cluster)
centroids = kmeans.cluster_centers_
print(centroids)

fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_cluster['time5'], df_cluster['time8'], df_cluster['time12'], c=kmeans.labels_.astype(float), s=50, alpha=0.5)
#ax.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
plt.show()

#pd.plotting.scatter_matrix(df_cluster, alpha=0.2, c=kmeans.labels_.astype(float), figsize=(15,15))
#plt.show()
# Matplotlib toolkits
# PCA
# Agglomeratives Clustern

In [ ]:


pca = PCA(n_components = 4 )
pca.fit(df_cluster)
df_pca = pca.transform(df_cluster)
df_pca = pd.DataFrame(df_pca, columns = ['P1', 'P2', 'P3', 'P4']) 
print(pca.score)


In [ ]:
pca.explained_variance_ratio_

In [ ]:
# Visualization before clustering

plt.scatter(df_pca.P1, df_pca.P2)
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')

In [ ]:

plt.figure(figsize = (10,7))
plt.title('Dendrograms')
plt.axhline(y=3, color='r', linestyle='--')
dend = sch.dendrogram(sch.linkage(df_pca, method='ward'))

In [ ]:
model = AgglomerativeClustering(n_clusters = 6)

y_means = model.fit_predict(df_pca)

In [ ]:


silhouette = silhouette_score(df_pca, y_means)
print('Silhouette Score: ', silhouette )

davies_bouldin = davies_bouldin_score(df_pca, y_means)
print('Davies Bouldin Score: ', davies_bouldin )



In [ ]:
# Visualization of the result

plt.figure(figsize=(10, 8))

plt.title('Agglomerative Clustering with 6 Clusters')

plt.scatter(df_pca.P1, df_pca.P2, c=y_means, s=100)